In [109]:
def diagram(ntv):
    diag = '---\ntitle: ' + ntv.ntv_name + '\n---\n' + ntv.ntv_type.long_name[1:]
    ntv_diag = Ntv.obj(ntv.ntv_value)
    for item in ntv_diag:
        if item.ntv_type.long_name == '$entity':
            for entity in item:
                diag += diag_entity(entity)
        if item.ntv_type.long_name == '$relationship':
            for relation in item:
                diag += diag_relation(relation)
    return diag

def diag_entity(entity):
    ent = '\n    ' + entity.ntv_name + ' {'
    for att in entity.ntv_value:
        ent += '\n        ' + att[0] + ' ' + att[1]
        if len(att) > 2:
            if att[2] in ('PK', 'FK', 'UK'):
                ent += ' ' + att[2]
            else:
                ent += ' "' + att[2] + '"'
        if len(att) > 3:
            ent += ' "' + att[3] + '"'
    return ent + '\n    }'

def diag_relation(relation):
    rel_left  = {'exactly one' : ' ||', 'zero or one': ' |o', 'zero or more': ' }o', 'one or more': ' }|'}
    rel_right = {'exactly one' : '|| ', 'zero or one': 'o| ', 'zero or more': 'o{ ', 'one or more': '|{ '}
    identif   = {'identifying' : '--', 'non-identifying' : '..'}
    
    att = relation.ntv_value
    ent = '\n    ' + att[0] + rel_left[att[1]] + identif[att[2]] + rel_right[att[3]] + att[4]
    if len(att) > 5:
        ent += ' : ' + att[5]
    return ent

In [110]:
order_example = { 
    'order example:$erDiagram' : { 
        '::$relationship': [ 
            [ 'CUSTOMER', 'exactly one', 'identifying', 'zero or more', 'ORDER',     'places'],
            [ 'ORDER',    'exactly one', 'identifying', 'one or more',  'LINE-ITEM', 'contains'] 
        ],
        '::$entity': {
            'CUSTOMER':  [ 
                ['string', 'name',         'PK', 'the name'], 
                ['string', 'custNumber'] 
            ], 
            'ORDER': [ 
                ['int',    'orderNumber',  'PK'],
                ['string', 'deliveryAdress'] 
            ],
            'LINE-ITEM': [ 
                ['string', 'productCode',  'PK'],
                ['int',    'quantity'],
                ['float',  'pricePerUnit'] 
            ]   
        },
     } }

In [111]:
from base64 import b64encode
from IPython.display import Image, display
from json_ntv import Ntv, NtvSet, NtvList, NtvSingle

ntv = Ntv.obj(order_example)
diag = diagram(ntv)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))